# In-Class Group Exercise Review | Solution


In [ ]:
# Group Review Exercise
import pandas as pd

%matplotlib inline

# Other imports as you see fit (sklearn, etc)

df = pd.read_csv('../datasets/titanic.csv')

# Print a basic summary of the data
print df.info()
df.head()

# Your objective:

*Given a set of features for a passenger (as defined in the dataset) predict whether or not they will survive on the Titanic.*


1. Perform any necessary conditioning or cleaning to the data
2. Define the train/test split
3. Choose and Define a model with all relevant parameters for that model
4. Fit the model
5. Determine the precision, recall, and F1 score for this model on the *training* dataset
6. Predict whether passengers with the  following features survive:

```
     pclass     sex   age  sibsp  parch    ticket      fare cabin embarked
          3  female   NaN      8      2   CA.2343   69.5500   NaN        S
          3    male   NaN      0      0    349214    7.8958   NaN        S
          3    male   NaN      0      0    349253    7.8958   NaN        C
          1    male  44.0      2      0     19928   90.0000   C78        Q
          2    male  31.0      0      0    244270   13.0000   NaN        S
          1  female  31.0      0      2     36928  164.8667    C7        S
          3  female  38.0      1      5    347077   31.3875   NaN        S
          2    male  30.0      0      0     28228   13.0000   NaN        S
          1    male   NaN      0      0   PC17757  227.5250   NaN        C
          2  female  17.0      0      0    SC1748   12.0000   NaN        C
```


---

# A Possible Solution:

### Step 1: Perform any necessary conditioning or cleaning to the data

In [ ]:
def preprocess_data(dataframe):
    is_male       = df['sex'].replace({'male': 1, 'female': 0})
    estimated_age = df['age'].fillna(df['age'].mean())
    
    is_male.rename("is_male", inplace=True)
    estimated_age.rename("estimated_age", inplace=True)

    df_temp = pd.concat([dataframe, is_male, estimated_age], axis=1).drop(['sex', 'age'], axis=1)
    
    return pd.get_dummies(df_temp, columns=['pclass', 'embarked'])

df_model = preprocess_data(df)

df_model.head(3)

In [ ]:
X = df_model.drop(['survived', 'name', 'ticket', 'cabin'], axis=1)
y = df_model['survived']  

print "Features:"
X.info();

---

### Step 2: Define the Train/Test Split

*We're just using a simple train/test split here. We can later use KFold for more robust modeling and validation later*

In [ ]:
from sklearn import tree, cross_validation, linear_model, metrics


---

### Step 3 & 4: Choose and Define a model with all relevant parameters for that model & fit the model

*We'll define a simple function to help us display model results*

In [ ]:
# This is just a utility method to help print a summary of the results
def format_model_results(sklearn_model, X_train, X_test, y_train, y_test):
    # Get the model's class name (E.g. LogisticRegression, KNearestNeighbor, etc.) and store it to a string
    model_class_name = sklearn_model.__class__.__name__  
    
    y_test_pred   = sklearn_model.predict(X_test)
    y_train_pred  = sklearn_model.predict(X_train)

    precision_score = metrics.precision_score(y_test_pred, y_test)
    recall_score    = metrics.recall_score(y_test_pred, y_test)
    f1_score        = metrics.f1_score(y_test_pred, y_test)
    f1_score_train  = metrics.f1_score(y_train_pred, y_train)
    auc_score       = metrics.roc_auc_score(y_test_pred, y_test)
    
    return pd.DataFrame({
            "Precision": precision_score, 
            "Recall": recall_score, 
            "F1 score": f1_score, 
            "F1 score (train)": f1_score_train, 
            "AUC": auc_score
        }, index=[model_class_name])

### <span style="color: blue;">Logistic Regression</span>

#### create a model and call it: logreg_model

In [ ]:
# Try additional models


### <span style="color: blue;">KNN</span>

#### create a model and call it: knn_model

### <span style="color: blue;">Decision Trees</span>

#### create a model and call it: dectree_model

### <span style="color: blue;">Random Forest Classifier</span>

#### create a model and call it: rf_model

## Comparison

In [ ]:
scores_df = pd.concat([
    format_model_results(logreg_model, X_train, X_test, y_train, y_test),
    format_model_results(knn_model, X_train, X_test, y_train, y_test),
    format_model_results(dectree_model, X_train, X_test, y_train, y_test),
    format_model_results(rf_model, X_train, X_test, y_train, y_test),
])

scores_df

### <span style="color: green; text-decoration: underline;">Note how simple it is to evaluate different models. Most of the work is in cleaning the data!</span>

---

# Solution Part II: Improving the model

This is a good start, but there are plenty of steps we can take to improve our results:

2. **Feature Engineering:** We may be able to add "synthetic" information to our dataset by adding features from information already present. For instance, we discarded name, but perhaps we could get more information about a passenger by searching for keywords relating to honorifics and titles like 'Sir', 'Miss', 'Mrs', etc. This is called **feature engineering**

1. **Parameter Tuning:** Modifying various parameters of our models may improve performance. For instance, for our RandomForest model we can control the complexity of our model by modifying `num_estimators`, `min_samples_leaf`, `max_features`, etc... *(See: http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)*

3. **Improved Cross Validation Techniques:** Because we're holding out 30% of our data for testing, we have less data samples on which to actually build our model. How can we address this?
> **Example:** Use K-Fold cross validation. A larger value of k will allow us to keep more of our data for training, but will require training the same model multiple times.

### Improvement 1: Feature Engineering

In our original analysis we dropped the `cabin`, `ticket`, and `name` columns. However, do these columns contain information that may improve our model?

We can apply our own intelligence to try to add information to the dataset which are not present in the original feature space. We call this *feature engineering*.

In [ ]:
##
# Get first character of values in the cabin column
#    See: http://pandas.pydata.org/pandas-docs/stable/text.html?highlight=string
# 
# Example: get_title("Braund, Mr. Owen Harris")  # -> will return 'mr'
def get_title(name):
    titles = ['miss.','mrs.','master.','mr.','dr.','rev.','sir.','col.','lady.','archibald','major.','capt.']
    
    if name:
        for title in titles:
            if name.lower().find(title.lower()) > -1:
                return title.replace(".", "")

    return "none"

def add_engineered_features(dataframe):
    honorifics         = dataframe['name'].map(get_title)
    cabin_prefixes     = dataframe['cabin'].str.get(0)
    ticket_is_numeric  = dataframe['ticket'].str.isnumeric()
    ticket_starts_with = dataframe['ticket'].str.get(0)

    honorifics.rename('honorific', inplace=True)
    cabin_prefixes.rename('cabin_prefix', inplace=True)
    ticket_is_numeric.rename('ticket_is_numeric', inplace=True)
    ticket_starts_with.rename('ticket_starts_with', inplace=True)

    df_processed = pd.concat([dataframe, honorifics, cabin_prefixes, ticket_is_numeric, ticket_starts_with], axis=1)
    df_processed = pd.get_dummies(df_processed, columns=['honorific', 'cabin_prefix', 'ticket_starts_with'])

    return df_processed.drop(['name', 'cabin', 'ticket'], axis=1)

# Clean the data:
df_model = preprocess_data(df)
df_model_improved = add_engineered_features(df_model)

X = df_model_improved.drop('survived', axis=1)
y = df_model_improved['survived']  


### Improvements 2 & 3: Evaluating the model with cross-validation and parameter tuning

In [ ]:
%%timeit
from sklearn.ensemble import RandomForestClassifier
n_folds = 20

#
#FILL HERE #
#

cv_score = cross_validation.cross_val_score(model_rf, X, y, cv=n_folds, scoring='roc_auc')

print "Avg AUC Score across %s folds: %s" % (n_folds, cv_score.mean())